In [1]:
#from games_setup import *
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import os
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

In [4]:
PREFIX = "BIOMD0000000"

def load_file_from_curated_data(num):

  format_num = format(num, '03d')
  filename = "%s%s_url.xml" % (PREFIX, format_num)
  path = os.path.join(cn.BIOMODELS_DIR, filename)
  simple = SimpleSBML()
  simple.initialize(path)
  return simple


In [5]:
simple = load_file_from_curated_data(167)

In [6]:
simple.reactions

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol,
 statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc,
 statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol,
 PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc,
 PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm,
 stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm,
 stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm]

In [7]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(167)
# for r in simple.reactions:
#   if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
#     print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

Model analyzed...
At least one error found.



In [8]:
dir(m)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adj',
 '_getNonBoundaryMolecules',
 '_getNonBoundaryReactions',
 '_node',
 '_pred',
 '_succ',
 'addArc',
 'addReaction',
 'addTypeOneError',
 'addTypeTwoError',
 'add_cycle',
 'add_edge',
 'add_edges_from',
 'add_node',
 'add_nodes_from',
 'add_path',
 'add_star',
 'add_weighted_edges_from',
 'adj',
 'adjacency',
 'adjlist_inner_dict_factory',
 'adjlist_outer_dict_factory',
 'analyze',
 'canceling_errors',
 'checkTypeOneError',
 'checkTypeTwoError',
 'clear',
 'convertMatrixToSOMReactions',
 'convertReactionToSOMReaction',
 'copy',
 'decomposeMatrix',
 'degree',


In [9]:
m.getStoichiometryMatrix(simple.reactions, simple.molecules)

,PstatDimerisation,statDephosphorylation,statPhosphorylation,PstatDimerisationNuc,PstatDimer__import,stat_export,stat_import
Pstat_sol,-2.0,0.0,1.0,0.0,0.0,0.0,0.0
PstatDimer_sol,1.0,0.0,0.0,0.0,-1.0,0.0,0.0
Pstat_nuc,0.0,-1.0,0.0,-2.0,0.0,0.0,0.0
stat_nuc,0.0,1.0,0.0,0.0,0.0,1.0,1.0
stat_sol,0.0,0.0,-1.0,0.0,0.0,-1.0,-1.0
species_test,0.0,0.0,1.0,0.0,0.0,0.0,0.0
PstatDimer_nuc,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [10]:
m.echelon_df

,statPhosphorylation,PstatDimerisation,PstatDimerisationNuc
{species_test},1.0,0.0,0.0
{Pstat_sol},1.0,-2.0,0.0
{PstatDimer_nuc=PstatDimer_sol},0.0,1.0,1.0
{Pstat_nuc=stat_nuc=stat_sol},-1.0,0.0,-2.0


In [11]:
m.echelon_df.T

,{species_test},{Pstat_sol},{PstatDimer_nuc=PstatDimer_sol},{Pstat_nuc=stat_nuc=stat_sol}
statPhosphorylation,1.0,1.0,0.0,-1.0
PstatDimerisation,0.0,-2.0,1.0,0.0
PstatDimerisationNuc,0.0,0.0,1.0,-2.0


In [12]:
m.rref_df

,statPhosphorylation,PstatDimerisation,PstatDimerisationNuc
{species_test},1.0,0.0,0.0
{Pstat_sol},0.0,-2.0,0.0
{PstatDimer_nuc=PstatDimer_sol},0.0,0.0,1.0
{Pstat_nuc=stat_nuc=stat_sol},0.0,2.0,-2.0


In [13]:
count = 0
for som in m.nodes:
  if len(som.molecules)>1:
    count = count + len(som.reactions)
print(count)

3


In [14]:
np.linalg.cond(m.lower_inverse)

1.0

In [15]:
len(simple.reactions)

7

In [16]:
resulting_series = gr.getResultingSeries("statPhosphorylation")

In [17]:
len(simple.reactions)

7

In [19]:
type1_report, error_num = gr.reportTypeOneError([m.type_one_errors[0]], explain_details=True)
print(type1_report)
print(error_num)

IndexError: list index out of range

In [ ]:
[m.type_three_errors[0]]

In [ ]:
a = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
a

In [ ]:
type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(type3_report)
print(error_num)

In [ ]:
type2_report, error_num = gr.reportTypeTwoError([m.type_two_errors[1]], explain_details=True)
print(type2_report)
print(error_num)

In [ ]:
type2_report

In [ ]:
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(echelon_report)
print(error_num)

In [ ]:
echelon_report

In [ ]:
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
print(canceling_report)
print(error_num)

In [ ]:
data_dir=cn.BIOMODELS_DIR
#files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
paths = [os.path.join(data_dir, filename) for filename in files]
# this will test which model has which error;;
model_count = 0
count = 0
simple = SimpleSBML
for file in files:
  try:
    simple.initialize(os.path.join(data_dir, file))
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    if m.analyze(error_details=False):
      if (len(simple.reactions) > 100) and (len(m.type_one_errors)>1):
        print(file, "has type I error with more than 100 reactions")
        count += 1
    if count > 5:
      break
  except:
    pass

In [ ]:
# this will test which model has which error;;
model_count = 0
count = 0
for i in range(1, 150):
  try:
    simple = load_file_from_curated_data(i)
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    if m.echelon_errors or m.type_three_errors:
        print("model", model_count, " has an echelon or type three error!")
        if count == 5:
          break
  except:
    pass